
## Bond Pricing Engines in QuantLib

QuantLib provides different bond pricing engines to value various types of bonds, such as zero-coupon, fixed-rate, and callable bonds. Bond pricing engines determine the present value of the bond’s cash flows based on yield curves, discount factors, and interest rate models.

In this notebook, we will cover:
- Pricing a Zero Coupon Bond using a Discounting Bond Engine
- Pricing a Fixed Rate Bond using a Discounting Bond Engine
- Introduction to Callable Bonds and Pricing with Embedded Options
    


### Example 1: Zero Coupon Bond Pricing

A zero-coupon bond pays no periodic interest; it is issued at a discount and matures at face value. To price it, we use the `DiscountingBondEngine`, which discounts the face value to present using a yield curve.
    

In [1]:

from QuantLib import *

# Setup for a zero coupon bond
maturity_date = Date(30, December, 2025)
face_amount = 100  # Face value of the bond

# Define a flat yield curve for discounting
today = Date(30, December, 2023)
Settings.instance().evaluationDate = today
rate = 0.05  # 5% interest rate
day_count = Actual360()
flat_curve = YieldTermStructureHandle(FlatForward(today, rate, day_count))

# Pricing Engine setup
discount_engine = DiscountingBondEngine(flat_curve)

# Zero coupon bond definition
zero_coupon_bond = ZeroCouponBond(0, TARGET(), face_amount, maturity_date, Following, 100)
zero_coupon_bond.setPricingEngine(discount_engine)

# Display bond price
zero_coupon_bond_price = zero_coupon_bond.cleanPrice()
zero_coupon_bond_price
    

90.38326014618372


### Example 2: Fixed Rate Bond Pricing

A fixed-rate bond pays periodic interest (coupons) and returns the face amount at maturity. Here, we use the `DiscountingBondEngine` to price a fixed-rate bond by discounting each coupon and the face value to the present.
    

In [2]:

# Setup for a fixed rate bond
issue_date = Date(30, December, 2023)
maturity_date = Date(30, December, 2030)
coupon_rate = 0.05  # 5% coupon rate
coupons = [coupon_rate]

# Fixed rate bond definition
schedule = Schedule(issue_date, maturity_date, Period(Annual), TARGET(), Following, Following, DateGeneration.Backward, False)
fixed_rate_bond = FixedRateBond(3, 100, schedule, coupons, Actual360(), Following, 100.0, issue_date)

# Attach the discount engine
fixed_rate_bond.setPricingEngine(discount_engine)

# Display bond price
fixed_rate_bond_price = fixed_rate_bond.cleanPrice()
fixed_rate_bond_price
    

99.24956859289853


### Example 3: Callable Bond Pricing (Overview)

Callable bonds include an option for the issuer to redeem the bond before maturity. In QuantLib, we can price callable bonds using a pricing engine that accounts for embedded options. While this example does not include callable bond pricing due to complexity, it introduces the concept.
    
For callable bond pricing, you may use the `TreeCallableFixedRateBondEngine`, which relies on interest rate models like `HullWhite` for option-adjusted valuations.
    


### Example 3: Callable Bond Pricing with Hull-White Model

Callable bonds allow the issuer to redeem the bond before its maturity date, giving them flexibility based on interest rate movements. In QuantLib, we can price callable bonds using the `TreeCallableFixedRateBondEngine`, which uses an interest rate model, such as the `HullWhite` model, to estimate the effect of the call option on bond pricing.

The following example illustrates callable bond pricing with:
- A Hull-White model for interest rates.
- A callable bond with annual coupons.
- The `TreeCallableFixedRateBondEngine` for option-adjusted valuation.
    

In [5]:
from QuantLib import HullWhite, TreeCallableFixedRateBondEngine, Callability, CallabilitySchedule, BondPrice, Schedule

# Define interest rate parameters for the Hull-White model
reversion_rate = 0.03   # Speed at which rates revert to the mean
volatility = 0.01       # Volatility of the interest rate

# Hull-White term structure based on the previously defined flat curve
hw_model = HullWhite(flat_curve, reversion_rate, volatility)

# Define a callable fixed-rate bond
issue_date = Date(30, December, 2023)
maturity_date = Date(30, December, 2030)
face_amount = 100
coupon_rate = 0.05
schedule = Schedule(issue_date, maturity_date, Period(Annual), TARGET(), Following, Following, DateGeneration.Backward, False)

# Define a call schedule allowing the issuer to call the bond at par on each coupon date
callability_schedule = CallabilitySchedule()
for date in schedule:
    callability_price = BondPrice(100.0, BondPrice.Clean)  # Set call price at par with BondPrice.Clean
    callability = Callability(callability_price, Callability.Call, date)
    callability_schedule.append(callability)

# Callable bond definition
callable_bond = CallableFixedRateBond(3, face_amount, schedule, [coupon_rate], Actual360(), Following, face_amount, issue_date, callability_schedule)

# Pricing engine using a tree-based method with Hull-White model
tree_engine = TreeCallableFixedRateBondEngine(hw_model, 100)
callable_bond.setPricingEngine(tree_engine)

# Price the callable bond
callable_bond_price = callable_bond.cleanPrice()
callable_bond_price


96.70199527594926


### Explanation of Callable Bond Pricing Setup

1. **Hull-White Model**: The Hull-White model is used to simulate interest rate paths, accounting for the mean reversion and volatility of rates.
2. **Callability Schedule**: Each date in the bond's schedule is assigned a call option, allowing the issuer to redeem the bond at par value on any coupon date.
3. **TreeCallableFixedRateBondEngine**: This engine, combined with the Hull-White model, calculates the option-adjusted value of the callable bond.
4. **Pricing**: The final clean price of the callable bond reflects its value with the embedded option considered.
    